In [355]:
import pandas as pd
import ast
import builtins
import os
from types import FunctionType

from PaladinEngine.archive.archive import Archive
from PaladinUI.paladin_cli.paladin_cli import dump_to_csv
from PaladinUI.paladin_server.paladin_server import PaladinServer

In [356]:
# CSV_FILE_PATHS = ["caterpillar_output_sdn.csv", "caterpillar_output_sn.csv"]
# CSV_FILE_PATHS = ["DiffTool/is_prime_naive.csv", "DiffTool/is_prime_square.csv"]
CSV_FILE_PATHS = ["is_prime_naive.csv", "is_prime_square.csv"]
# CSV_FILE_PATHS = ["is_prime_naive.csv"]
TEST_CSV_FILE_PATH_TEMPLATE = "DiffTool/test_{}"

In [357]:
def create_archive_from_csv(csv_path):
    dataframe = load_csv_to_dataframe(csv_path)
    archive_from_csv = Archive()
    rows = dataframe.to_records()
    for row in rows: #TODO: look for a builtin function instead of loop?
        record_key, record_value = create_record_key_value(row)
        archive_from_csv.store_with_original_time(record_key, record_value)
    return archive_from_csv

def load_csv_to_dataframe(csv_path):
    dataframe = pd.read_csv(csv_path)
    dataframe = dataframe.fillna('')
    return dataframe

def create_record_key_value(row):
    record_key = Archive.Record.RecordKey(int(row.container_id), row.field, row.stub_name)

    if row.rtype == 'function':
        value_of_record = row.value
        type_of_record = FunctionType
    elif row.rtype == 'list':
        value_of_record = ast.literal_eval(row.value)
        type_of_record = list
    else:
        value_of_record = getattr(builtins, row.rtype)(row.value)
        type_of_record = type(value_of_record)

    record_value = Archive.Record.RecordValue(
        record_key, type_of_record, value_of_record, row.expression,
        int(row.line_no), int(row.time), row.extra
    )
    return record_key, record_value

In [358]:
def convert_query_result_to_presentable_table(query_result):
    result = query_result['result']['query']
    # Change shape so time ranges are rows instead of columns
    result_df = pd.read_json(result).transpose()
    # Extract only needed columns and rename them
    result_df.reset_index(inplace=True)
    times_column_name = 'Time Range'
    result_df = result_df.rename(columns={'index': times_column_name})
    first_column = result_df[0]
    first_value = first_column.iat[0]
    queries_from_result = list(first_value.keys())
    for query_from_result in queries_from_result:
        result_df[query_from_result] = first_column.apply(lambda d: d.get(query_from_result))
    subset_columns = [times_column_name] + queries_from_result
    minimal_df = result_df[subset_columns]
    return minimal_df, queries_from_result

In [359]:
def get_parameter_matches_with_previous_query(parameters, csv_path):
    parameter_matches = {}
    for parameter in parameters:
        parameter_matches[parameter] = input(f"Match {parameter} in {csv_path}: ")
    return parameter_matches

def replace_matched_parameters(previous_query, parameter_matches):
    matched_query = previous_query
    for source_parameter, dest_parameter in parameter_matches.items():
        matched_query = matched_query.replace(source_parameter, dest_parameter)
    return matched_query

def convert_parameter_to_queryable(parameter):
    location_separator = '@'
    if location_separator in parameter:
        parameter_name, parameter_location = parameter.split(location_separator)
        queryable = f"[[{parameter_name}]]@{parameter_location}"
    else:
        queryable = f"[[{parameter}]]"
    return queryable

In [360]:
def main():
    # Load the csv file into a DataFrame object
    archives = []
    initial_query = input("Enter your query: ")
    initial_parameters = []
    for index, csv_path in enumerate(CSV_FILE_PATHS):
        csv_archive = create_archive_from_csv(csv_path)
        archives.append(csv_archive)

        # Run a query using the archive created from the csv file
        server = PaladinServer.create('', csv_archive)

        is_initial_query = index == 0
        matched_query = initial_query
        if not is_initial_query:
            parameter_matches = get_parameter_matches_with_previous_query(initial_parameters, csv_path)
            matched_query = replace_matched_parameters(initial_query, parameter_matches)
        # raw_result = server.query('[[i]]@4', 0, 500, 0)
        # raw_result = server.query('Union([[i]]@4, [[number]]@11)', 0, 500, 0)
        raw_result = server.query(matched_query, 0, 500, 0)

        # Convert the result to the required form
        presentable_df, parameters = convert_query_result_to_presentable_table(raw_result)
        if is_initial_query:
            initial_parameters = [convert_parameter_to_queryable(parameter) for parameter in parameters]
        print(presentable_df)
        # dump_to_csv(csv_archive, TEST_CSV_FILE_PATH_TEMPLATE.format(os.path.basename(csv_path))) #TODO: remove later

if __name__ == '__main__':
    main()

     Time Range    i@4  number@11
0        (0, 1)    NaN        NaN
1        (2, 6)    NaN      173.0
2        (7, 7)    2.0      173.0
3        (8, 8)    3.0      173.0
4        (9, 9)    4.0      173.0
..          ...    ...        ...
168  (173, 173)  168.0      173.0
169  (174, 174)  169.0      173.0
170  (175, 175)  170.0      173.0
171  (176, 176)  171.0      173.0
172  (177, 500)  172.0      173.0

[173 rows x 3 columns]
   Time Range   i@8  number@15
0      (0, 1)   NaN        NaN
1      (2, 8)   NaN      173.0
2      (9, 9)   2.0      173.0
3    (10, 10)   3.0      173.0
4    (11, 11)   4.0      173.0
5    (12, 12)   5.0      173.0
6    (13, 13)   6.0      173.0
7    (14, 14)   7.0      173.0
8    (15, 15)   8.0      173.0
9    (16, 16)   9.0      173.0
10   (17, 17)  10.0      173.0
11   (18, 18)  11.0      173.0
12   (19, 19)  12.0      173.0
13  (20, 500)  13.0      173.0


In [361]:
"""
Run:
python C:\Avital\Github\paladin_engine\PaladinUI\paladin_cli\paladin_cli.py --run --output-file output.py --csv DiffTool\is_prime_naive.csv --run-debug-server True --port 1234
C:\Avital\Github\paladin_engine\PaladinEngine\tests\test_resources\examples\is_prime\is_prime_naive.py

#TODO:
1) keys id: update the ctor
2) time: 0 is converted to [1,5] (line_no==85)
3) set: not supported

#TODO: 28.11
1) keys: id: update the ctor? ask Oren why it is needed id(v.key)
2) store: write our function which doesn't change time
3) represent: call represent asap (maybe in ctor) instead of in to_table
4) paladin_server.py: debug_info/query/ - check that the Archive is OK

#Questions:
1) Will we always have exactly 2 tables to compare, or can there be more?
2) How exactly are we supposed to connect each two tables? For example in is_prime,
should we just match the 13 rows of 'square' to the first 13 rows of 'naive'?
3) User input - how? Using python's input(), or maybe read from file?
Maybe should depend on the number of parameters in the query

"""

"\nRun:\npython C:\\Avital\\Github\\paladin_engine\\PaladinUI\\paladin_cli\\paladin_cli.py --run --output-file output.py --csv DiffTool\\is_prime_naive.csv --run-debug-server True --port 1234\nC:\\Avital\\Github\\paladin_engine\\PaladinEngine\tests\test_resources\\examples\\is_prime\\is_prime_naive.py\n\n#TODO:\n1) keys id: update the ctor\n2) time: 0 is converted to [1,5] (line_no==85)\n3) set: not supported\n\n#TODO: 28.11\n1) keys: id: update the ctor? ask Oren why it is needed id(v.key)\n2) store: write our function which doesn't change time\n3) represent: call represent asap (maybe in ctor) instead of in to_table\n4) paladin_server.py: debug_info/query/ - check that the Archive is OK\n\n#Questions:\n1) Will we always have exactly 2 tables to compare, or can there be more?\n2) How exactly are we supposed to connect each two tables? For example in is_prime,\nshould we just match the 13 rows of 'square' to the first 13 rows of 'naive'?\n3) User input - how? Using python's input(), or